## 1D__CNN and DataEnhancement (pitch shift)

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras import optimizers
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from keras import regularizers
import librosa

Using TensorFlow backend.


In [2]:
# import competition dataset but downsampling to 3999*4410

x_train = np.load('train_data.npy')
y_train = pd.read_csv('train_labels.csv')
y_train = np.array(y_train)
y_train = y_train[:,1]

x_test = np.load('test_data.npy')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [3]:
# DATA_ENHANCE_x_train

array_ts = []
for i in range(len(x_train)):
    y_ts = librosa.effects.pitch_shift(x_train[i],x_train.shape[1], n_steps=3)
    array_ts.append(y_ts)
x_ts= np.vstack(array_ts)
print(x_ts.shape)

# DATA_ENHANCE___x_test

array_tsTest = []
for i in range(len(x_test)):
    y_tsTest = librosa.effects.pitch_shift(x_test[i],x_test.shape[1], n_steps=3)
    array_tsTest.append(y_tsTest)
x_tsTest = np.vstack(array_tsTest)
print(x_tsTest.shape)

# reshape
x_train = x_ts.reshape(-1,x_ts.shape[1],1)
x_test = x_tsTest.reshape(-1,x_tsTest.shape[1],1)

print(x_train.shape,x_test.shape,y_train.shape)

(3999, 44100)
(3997, 44100)
(3999, 44100, 1) (3997, 44100, 1) (3999,)


In [4]:
# make output a 2 dim vector indicating class
y_train = np_utils.to_categorical(y_train, 2)

In [17]:
# create 1-D CNN
model = Sequential()
model.add(Conv1D(32, kernel_size=5, 
                 activation='relu',
                 input_shape=(x_train.shape[1],1),
                 kernel_regularizer=regularizers.l1(0.001))) 
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=50,strides=10))

model.add(Conv1D(64, kernel_size=5, 
                 activation='relu',
                 kernel_regularizer=regularizers.l1(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=50,strides=10))

model.add(Conv1D(128, kernel_size=5, 
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=50,strides=10))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


In [18]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, 44096, 32)         192       
_________________________________________________________________
batch_normalization_9 (Batch (None, 44096, 32)         128       
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 4405, 32)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 4401, 64)          10304     
_________________________________________________________________
batch_normalization_10 (Batc (None, 4401, 64)          256       
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 436, 64)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 432, 128)         

In [19]:
# compile model
adam=optimizers.Adam(lr=.0001) 
model.compile(loss='binary_crossentropy', 
              optimizer=adam,
              metrics=['accuracy'])

In [20]:
from keras.callbacks import EarlyStopping
EarlyStopping = EarlyStopping(monitor='val_loss',patience=5,verbose=0,mode='auto')
model.fit(x_train, y_train, batch_size=32, epochs=30, validation_split=0.1,callbacks=[EarlyStopping])


Train on 3599 samples, validate on 400 samples
Epoch 1/30
3599/3599 [==============================] - 457s 127ms/step - loss: 0.9624 - acc: 0.9241 - val_loss: 0.7709 - val_acc: 0.9450
Epoch 2/30
3599/3599 [==============================] - 916s 255ms/step - loss: 0.7628 - acc: 0.9467 - val_loss: 0.7288 - val_acc: 0.9525
Epoch 3/30
3599/3599 [==============================] - 781s 217ms/step - loss: 0.7281 - acc: 0.9550 - val_loss: 0.7062 - val_acc: 0.9650
Epoch 4/30
3599/3599 [==============================] - 775s 215ms/step - loss: 0.6843 - acc: 0.9661 - val_loss: 0.6574 - val_acc: 0.9725
Epoch 5/30
3599/3599 [==============================] - 833s 231ms/step - loss: 0.6567 - acc: 0.9667 - val_loss: 0.6651 - val_acc: 0.9600
Epoch 6/30
3599/3599 [==============================] - 790s 219ms/step - loss: 0.6515 - acc: 0.9680 - val_loss: 0.6517 - val_acc: 0.9625
Epoch 7/30
3599/3599 [==============================] - 777s 216ms/step - loss: 0.6378 - acc: 0.9689 - val_loss: 0.6306 - val

In [ ]:
# make predictions
y_predict= model.predict_classes(x_test)
print(y_predict[:500])

In [ ]:
print(x_test.shape)

In [ ]:
prediction = pd.DataFrame(y_predict,columns=['label']).to_csv('submission_1dCNN.csv')

In [ ]:
"TrainingNote:"
// loss: 0.1601 - acc: 0.9372 - val_loss: 0.1614 - val_acc: 0.9475
//loss: 0.2104 - acc: 0.9564 - val_loss: 0.1292 - val_acc: 0.9650
// add a convlayer 128 // loss: 0.0484 - acc: 0.9844 - val_loss: 0.0922 - val_acc: 0.9675
// add l1 on the first two conv layers //loss: 0.4179 - acc: 0.9839 - val_loss: 0.4799 - val_acc: 0.9775